# Chapter 26: Smart Contract Security

---

Smart contract security is not an afterthought—it is the foundation upon which all decentralized applications must be built. In traditional software, a bug might cause inconvenience; in smart contracts, a bug can lead to irreversible loss of funds. This chapter delves deep into the security mindset, common vulnerabilities, best practices, tools, and the audit process. By understanding both the theory and practical countermeasures, you will be equipped to write robust contracts that can withstand attacks.

---

## 26.1 Security Mindset

### 26.1.1 Immutable Code, Permanent Bugs

Once a smart contract is deployed on the blockchain, its code cannot be changed (unless using upgradeable patterns, which themselves introduce complexity). This immutability means that any vulnerability present at deployment is there forever, waiting to be exploited.

**Key implications:**
- **No hotfixes**: You cannot push a quick patch; you must convince users to migrate to a new contract.
- **Economic incentives**: Attackers are financially motivated to find and exploit bugs.
- **Public code**: Your code is visible to everyone, including attackers.

Therefore, security must be baked into every stage of development, from design to deployment.

### 26.1.2 Value at Risk

Smart contracts often control significant financial value. Even a small bug can be catastrophic.

**Historical losses:**
- **The DAO (2016)**: 3.6 million ETH (≈ $60M at the time) stolen due to reentrancy.
- **Parity Wallet (2017)**: $30M frozen due to library self-destruct.
- **Numerous DeFi hacks**: Hundreds of millions lost to flash loan attacks, oracle manipulation, etc.

This risk demands rigorous testing, auditing, and a security-first culture.

---

## 26.2 Common Vulnerabilities

Understanding the most common vulnerabilities is the first step toward avoiding them. We'll examine each with examples and mitigations.

### 26.2.1 Reentrancy

Reentrancy occurs when a contract calls an external contract before updating its own state, allowing the external contract to recursively call back into the original function and repeat state-changing operations.

**Vulnerable code:**
```solidity
contract Vulnerable {
    mapping(address => uint) public balances;

    function withdraw() public {
        uint bal = balances[msg.sender];
        require(bal > 0);

        (bool success, ) = msg.sender.call{value: bal}(""); // external call
        require(success);

        balances[msg.sender] = 0; // state update AFTER call
    }
}
```

**Attack:**
- Attacker's contract has a `receive()` function that calls `withdraw()` again.
- Since balance hasn't been set to 0 yet, the second call succeeds, draining funds.

**Mitigation:**
- **Checks-Effects-Interactions**: Update state before external calls.
- **Reentrancy Guard**: Use OpenZeppelin's `ReentrancyGuard` modifier.

```solidity
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";

contract Secure is ReentrancyGuard {
    mapping(address => uint) public balances;

    function withdraw() public nonReentrant {
        uint bal = balances[msg.sender];
        require(bal > 0);
        balances[msg.sender] = 0; // update first
        (bool success, ) = msg.sender.call{value: bal}("");
        require(success);
    }
}
```

### 26.2.2 Access Control Issues

Functions that should be restricted to certain roles (e.g., owner) but are left public can be called by anyone.

**Vulnerable code:**
```solidity
contract Crowdsale {
    address public owner;
    bool public paused;

    function pause() public { // missing onlyOwner
        paused = true;
    }
}
```

**Mitigation:**
- Use modifiers like `onlyOwner` from OpenZeppelin's `Ownable`.
- Use role-based access control (RBAC) with `AccessControl`.

```solidity
import "@openzeppelin/contracts/access/Ownable.sol";

contract Crowdsale is Ownable {
    bool public paused;
    function pause() public onlyOwner {
        paused = true;
    }
}
```

### 26.2.3 Integer Overflow/Underflow

Before Solidity 0.8.x, arithmetic operations could wrap around (e.g., `uint8(255) + 1 = 0`). Since 0.8.x, overflow/underflow causes a revert by default, but unchecked blocks can still cause issues.

**Vulnerable code (pre-0.8):**
```solidity
uint8 balance = 255;
balance += 1; // becomes 0
```

**Mitigation:**
- Use Solidity 0.8.x or later.
- Use SafeMath library for older versions.
- Be cautious with `unchecked` blocks (only use when overflow is impossible or desired).

### 26.2.4 Unchecked Return Values

External calls often return a boolean indicating success. Ignoring this can lead to false assumptions.

**Vulnerable code:**
```solidity
token.transfer(msg.sender, amount); // returns bool, ignored
// If transfer fails, execution continues incorrectly
```

**Mitigation:**
- Always check return values.
- Use OpenZeppelin's `SafeERC20` which reverts on failure.

```solidity
import "@openzeppelin/contracts/token/ERC20/utils/SafeERC20.sol";
using SafeERC20 for IERC20;

token.safeTransfer(msg.sender, amount);
```

### 26.2.5 Denial of Service (DoS)

Attackers can make a contract unusable by, for example, causing an array to grow too large (gas limit), or making a function always revert.

**Examples:**
- **Unbounded loop**: Iterating over a dynamic array that can be enlarged by anyone.
- **Unexpected revert**: Making a `receive()` function that always reverts, blocking ETH transfers.

**Mitigation:**
- Avoid loops over dynamic arrays that can grow unbounded.
- Use pull over push for payments (let users withdraw instead of pushing to them).
- Implement circuit breakers (pause) to stop functionality in emergencies.

### 26.2.6 Front-Running (MEV)

Transactions are visible in the mempool before being mined. Attackers can observe pending transactions and submit their own with higher gas to get executed first, exploiting the original transaction.

**Example:** A user submits a profitable trade; a miner sees it and front-runs it with a higher gas price, capturing the profit.

**Mitigation:**
- Use commit-reveal schemes (user commits to a hash, later reveals).
- Set slippage tolerance in DEX trades.
- Use private mempools (Flashbots, etc.).

### 26.2.7 Oracle Manipulation

If a contract relies on a single source for price data, an attacker can manipulate that source (e.g., a low-liquidity DEX pool) to trigger unfair liquidations or trades.

**Example:** An attacker uses a flash loan to manipulate a DEX price, then triggers a liquidation in a lending protocol that uses that DEX as an oracle.

**Mitigation:**
- Use decentralized oracles (Chainlink) with multiple sources.
- Use time-weighted average prices (TWAP) from DEXes.
- Implement circuit breakers (e.g., reject prices that deviate too much from previous).

### 26.2.8 Flash Loan Attacks

Flash loans allow borrowing large sums without collateral within a single transaction. Attackers combine flash loans with other vulnerabilities (price manipulation, reentrancy) to drain protocols.

**Mitigation:**
- Avoid relying on spot prices from easily manipulated pools.
- Use TWAP or decentralized oracles.
- Implement borrow caps and sanity checks.

### 26.2.9 Signature Replay

If signatures are not protected against replay across different chains or contracts, an attacker can reuse a signature to execute unauthorized actions.

**Example:** A signature for a token permit (ERC-2612) on Ethereum mainnet could be replayed on a sidechain if the contract doesn't include chain ID in the signed message.

**Mitigation:**
- Include `chainId` and contract address in the signed message (EIP-712).
- Use nonces to prevent replay of the same signature.

### 26.2.10 Delegate Call Vulnerabilities

`delegatecall` executes code in the context of the calling contract. If the target contract is malicious or has a self-destruct, the caller's contract can be destroyed or its state corrupted.

**Example:** A proxy contract that delegates to an implementation that self-destructs will cause the proxy to self-destruct as well.

**Mitigation:**
- Never `delegatecall` into untrusted contracts.
- Ensure implementation contracts cannot self-destruct or change owner in harmful ways.
- Use OpenZeppelin's upgradeable contracts which handle these safely.

---

## 26.3 Security Best Practices

Adopting a set of best practices reduces the likelihood of introducing vulnerabilities.

### 26.3.1 Checks-Effects-Interactions Pattern

This pattern ensures that all state changes happen before any external calls. It prevents reentrancy and other race conditions.

```solidity
function withdraw() public {
    // Checks
    uint bal = balances[msg.sender];
    require(bal > 0);
    
    // Effects
    balances[msg.sender] = 0;
    
    // Interactions
    (bool success, ) = msg.sender.call{value: bal}("");
    require(success);
}
```

### 26.3.2 Pull Over Push Pattern

Instead of sending funds to users (push), let users withdraw their own funds (pull). This avoids DoS where a malicious receiver reverts.

```solidity
mapping(address => uint) public pendingWithdrawals;

function withdraw() public {
    uint amount = pendingWithdrawals[msg.sender];
    pendingWithdrawals[msg.sender] = 0;
    payable(msg.sender).transfer(amount);
}
```

### 26.3.3 Circuit Breaker (Pausable)

Allow an admin to pause the contract in case of an emergency, stopping critical functions until the issue is resolved.

```solidity
import "@openzeppelin/contracts/security/Pausable.sol";

contract MyContract is Pausable {
    function transfer(address to, uint amount) public whenNotPaused {
        // ...
    }
}
```

### 26.3.4 Rate Limiting

Prevent rapid repeated calls that could drain funds or cause abuse. For example, limit withdrawals per time period.

```solidity
mapping(address => uint) lastWithdrawTime;
uint constant cooldown = 1 days;

function withdraw() public {
    require(block.timestamp >= lastWithdrawTime[msg.sender] + cooldown);
    lastWithdrawTime[msg.sender] = block.timestamp;
    // ...
}
```

### 26.3.5 Access Control Patterns

Use well-tested access control libraries:

- **Ownable**: Simple single-owner.
- **Roles**: OpenZeppelin's `AccessControl` for multi-role systems.
- **Timelocks**: Delay sensitive actions (e.g., ownership transfer) to give users time to react.

---

## 26.4 Security Tools

Automated tools can catch many common vulnerabilities before human review.

### 26.4.1 Static Analysis (Slither, Mythril)

Static analyzers examine source code without executing it, looking for known vulnerability patterns.

**Slither** (by Trail of Bits):
- Detects reentrancy, unused variables, incorrect access control, etc.
- Integrates with CI.

```bash
slither .
```

**Mythril**:
- Symbolic execution and taint analysis.
- Can detect more complex vulnerabilities.

```bash
myth analyze contract.sol
```

### 26.4.2 Symbolic Execution (Manticore)

Symbolic execution explores multiple execution paths to find bugs. It's more powerful but slower than static analysis.

**Manticore** (by Trail of Bits):
- Can find assertion violations, integer overflows, etc.

```python
from manticore.ethereum import ManticoreEVM
m = ManticoreEVM()
# ... analyze contract
```

### 26.4.3 Formal Verification

Mathematically proves that a contract satisfies certain properties. Used for critical systems (e.g., the Ethereum deposit contract).

Tools: **Certora Prover**, **KEVM**, **Why3**.

### 26.4.4 Fuzzing Tools

As covered in Chapter 25, fuzzing (Echidna, Foundry) generates random inputs to test invariants. Essential for finding edge cases.

---

## 26.5 Audits

An audit is a thorough security review by external experts. It's not a guarantee of safety but significantly reduces risk.

### 26.5.1 When to Get an Audit

- Before mainnet deployment.
- After significant changes to the code.
- For high-value or critical contracts.
- Even for seemingly simple contracts (bugs can hide anywhere).

### 26.5.2 Audit Process

A typical audit follows these phases:

1. **Scope definition**: Client and auditor agree on which contracts and commit hash to review.
2. **Initial review**: Auditor examines code, runs tools, and documents findings.
3. **Fixes and retesting**: Client fixes issues; auditor verifies fixes.
4. **Final report**: Detailed report with findings, severity ratings, and recommendations.

**Severity ratings:**
- **Critical**: Direct loss of funds, must fix.
- **High**: Significant risk, should fix.
- **Medium**: Potential issue under specific conditions.
- **Low**: Informational or minor.

**Choosing an auditor:** Look for reputation, experience, and past audits. Popular firms: Trail of Bits, ConsenSys Diligence, OpenZeppelin, Quantstamp, CertiK.

### 26.5.3 Bug Bounty Programs

After audit, consider a bug bounty program to incentivize the community to find remaining issues. Platforms: Immunefi, HackerOne.

**Setting up a bounty:**
- Define scope and rewards (e.g., up to $1M for critical bugs).
- Provide clear rules and reporting process.
- Use an escrow or trusted platform.

---

## 26.6 Incident Response

Despite all precautions, incidents can still happen. Having a plan can mitigate damage.

### 26.6.1 Preparing for Hacks

- **Emergency pause**: As mentioned, have a circuit breaker.
- **Upgradeability**: If your contract is upgradeable, you can fix bugs (but this centralizes trust).
- **Monitoring**: Set up alerts for unusual activity (e.g., large transfers, oracle deviations).
- **Communication channels**: Have a plan to notify users quickly (Twitter, Discord, etc.).

### 26.6.2 Post-Mortem Analysis

After an incident, conduct a thorough analysis:

- **What happened?** Describe the attack.
- **Why did it happen?** Root cause (vulnerability, misconfiguration).
- **Impact**: Funds lost, users affected.
- **Response**: Steps taken (pause, upgrade, recovery).
- **Prevention**: How to avoid similar issues in the future.

Transparency builds trust. Publish a post-mortem.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 26 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Security is paramount due to immutability and financial risk.  │
│                                                                 │
│  Common vulnerabilities:                                        │
│    • Reentrancy (use Checks-Effects-Interactions, guard)       │
│    • Access control (use Ownable, AccessControl)               │
│    • Integer overflow (use 0.8.x, SafeMath)                    │
│    • Unchecked return values (use SafeERC20)                   │
│    • DoS (avoid unbounded loops, pull over push)               │
│    • Front-running (commit-reveal, slippage)                   │
│    • Oracle manipulation (use decentralized oracles, TWAP)     │
│    • Flash loan attacks (sanity checks, TWAP)                  │
│    • Signature replay (include chainId, nonce)                 │
│    • delegatecall vulnerabilities (avoid untrusted targets)    │
│                                                                 │
│  Best practices:                                                │
│    • Checks-Effects-Interactions                               │
│    • Pull over push                                            │
│    • Circuit breakers                                          │
│    • Rate limiting                                             │
│    • Well-tested access control                                │
│                                                                 │
│  Tools: Slither, Mythril, Manticore, Echidna, Certora.         │
│  Audits and bug bounties are essential.                        │
│  Have an incident response plan.                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 27 – Gas Optimization. We'll explore techniques to reduce gas costs, from storage packing to efficient data structures and inline assembly.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='25. smart_contract_testing.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='27. gas_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
